In [1]:
!pip install vosk pydub transformers pyaudio ipywidgets torch -f https://download.pytorch.org/whl/torch_stable.html

#it took me 11 minutes 50 seconds to run local machine . i.e normal cpu

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 5.2 MB/s eta 0:07:50
     ---------------------------------------- 0.0/2.5 GB 4.9 MB/s eta 0:08:21
     ---------------------------------------- 0.0/2.5 GB 4.9 MB/s eta 0:08:25
     ---------------------------------------- 0.0/2.5 GB 5.2 MB/s eta 0:07:49
     ---------------------------------------- 0.0/2.5 GB 5.1 MB/s eta 0:07:58
     ---------------------------------------- 0.0/2.5 GB 4.9 MB/s eta 0:08:18
     ---------------------------------------- 0.0/2.5 GB 5.0 MB/s eta 0:08:16
     ---------------------------------------- 0.0/2.5 GB 4.9 MB/s eta 0:08:18
     ---------------------------------------- 0.0/2.5 GB 4.9 MB/s eta 0:08:24
     -----------------------------


     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:20
     ---------- ----------------------------- 0.6/2.5 GB 4.8 MB/s eta 0:06:20
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:20
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.8 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.8 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.8 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB/s eta 0:06:19
     ---------- ----------------------------- 0.6/2.5 GB 4.9 MB

In [1]:
import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

In [14]:
!pip install portaudio

ERROR: Could not find a version that satisfies the requirement portaudio (from versions: none)
ERROR: No matching distribution found for portaudio


In [15]:
!pip install pyaudio

In [16]:
# we can install pyaudio from http://people.csail.mit.edu/hubert/pyaudio/
# we can see our audio devices index using this code
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microphone Array (Realtek(R) Au', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannel

In [17]:
CHANNELS = 1  #speech 
FRAME_RATE = 16000   #default for speech recognization for good frequency
RECORD_SECONDS = 20   #we are giving 20 seconds time to record
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [18]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")    #it takes around 10 minutes to run 
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased)
        time.sleep(1)

In [19]:
# Find audio device index
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microphone Array (Realtek(R) Au', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannel

Exception in thread Thread-26 (speech_recognition):
Traceback (most recent call last):
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\poppo\AppData\Local\Temp\ipykernel_31492\3066916822.py", line 19, in speech_recognition
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command 'python recasepunc/recasepunc

In [20]:
from vosk import Model
Model(model_name="vosk-model-small-en-us-0.15")

vosk-model-small-en-us-0.15.zip: 100%|██████████| 39.3M/39.3M [00:30<00:00, 1.35MB/s]  


Exception in thread Thread-29 (speech_recognition):
Traceback (most recent call last):
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\poppo\AppData\Local\Temp\ipykernel_31492\3066916822.py", line 19, in speech_recognition
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command 'python recasepunc/recasepunc

In [22]:
from vosk import Model
import os

model_path = "vosk-model-small-en-us-0.15"
root_path = r"C:\Users\poppo\Desktop\github ai\end to end by vivek\repo-46-speech-to-text-model"

# Download and save the model to the root directory
model = Model(model_name=model_path, root=root_path)


Exception ignored in: <function Model.__del__ at 0x000001FDD4BAB4C0>
Traceback (most recent call last):
  File "c:\Users\poppo\anaconda3\envs\genai\Lib\site-packages\vosk\__init__.py", line 60, in __del__
    _c.vosk_model_free(self._handle)
                       ^^^^^^^^^^^^
AttributeError: 'Model' object has no attribute '_handle'


TypeError: Model.__init__() got an unexpected keyword argument 'root'